In [1]:
import pandas as pd
import numpy as np
import constants
import data_extraction
if constants.run_data_expectations:
    from expectations_helpers import (
        AutoGreatExpectations,
        view_full_suite,
        view_suite_summary,
        save_expectations,
        load_expectations,
        validate_data
    )
import preprocessing
import joblib
import prediction
from tabulate import tabulate
import runpy
import visualisations
import shap
import matplotlib.pyplot as plt
import training
import evaluation
import gc

In [2]:
import importlib
importlib.reload(prediction)
importlib.reload(visualisations)
importlib.reload(constants)
importlib.reload(data_extraction)
importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/david@inawisdom.com/Documents/Training/premier_league/src/preprocessing.py'>

In [3]:
full_data = pd.read_csv(constants.save_location)

### Preprocessing

In [4]:
transformers = joblib.load(constants.transformer_path)

In [5]:
transformed_data = preprocessing.transform_data(
    full_data, transformers
)

### Training Loop

In [6]:
target_column = 'FTR'

In [7]:
transformed_data.head()

,season,HomeTeam,AwayTeam,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,MW,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,HomeLeaguePosition,AwayLeaguePosition,LeaguePositionDiff,MatchDayDay,MatchDayMonth,MatchDayDate
0,1995-96,Aston Villa,Man United,0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0,0,0.0,0.0,0.0,0.0,2,11,-9,Saturday,August,19
1,1995-96,Blackburn,QPR,0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0,0,0.0,0.0,0.0,0.0,3,15,-12,Saturday,August,19
2,1995-96,Chelsea,Everton,1,0.0,0.0,0.0,0.0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0,0,0.0,0.0,0.0,0.0,5,7,-2,Saturday,August,19
3,1995-96,Liverpool,Sheffield Weds,0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0,0,0.0,0.0,0.0,0.0,9,16,-7,Saturday,August,19
4,1995-96,Man City,Tottenham,1,0.0,0.0,0.0,0.0,0.0,0.0,M,M,M,M,M,M,M,M,M,M,1.0,0,0,0.0,0.0,0.0,0.0,10,18,-8,Saturday,August,19


In [8]:
evaluation_accuracy = pd.DataFrame()
for season in sorted(transformed_data['season'].unique()):
    for mw in sorted(transformed_data['MW'].unique()):
        print(season, mw)
        transformed_data['season'] = transformed_data['season'].astype(str)
        transformed_data_mini =  transformed_data[(transformed_data[
            'season'] <= season) & (transformed_data[
            'MW'] <= mw)]
        transformed_data_mini['season'] = transformed_data_mini[
            'season'].astype('category')
        target_column = 'FTR'
        training_data, testing_data = preprocessing.split_data(
            transformed_data_mini)
        hyperparameters = training.optimise_hyperparameters(
            training_data,target_column,
            max_evals=5
        )
        classifier = training.train_model(
            training_data[[
                col for col in training_data if col != target_column]],
            training_data[target_column],
            hyperparameters = hyperparameters
        )
        y_test = testing_data[target_column]
        x_test = testing_data[[col for col in testing_data if col != target_column]]
        predictions = prediction.predict(x_test, classifier)
        
        evaluation_metrics = evaluation.evaluate_model(predictions, y_test)
        print(evaluation_metrics)
        evaluation_acc_mini = pd.DataFrame()
        evaluation_acc_mini['season'] = [season]
        evaluation_acc_mini['MW'] = [mw]
        evaluation_acc_mini['mcc'] = [evaluation_metrics['mcc']]
        evaluation_acc_mini['accuracy'] = [evaluation_metrics['accuracy']]
        evaluation_acc_mini['f1'] = [evaluation_metrics['f1']]
        evaluation_accuracy = pd.concat([evaluation_accuracy, evaluation_acc_mini])
        del evaluation_acc_mini
        del transformed_data_mini
        del training_data
        del testing_data
        del y_test
        del predictions
        del classifier
        del evaluation_metrics
        gc.collect()
        

1995-96 1.0
Entering fmin
CatBoost initialized                                                            
Fitting model                                                                   
Loss: -0.5                                                                      
CatBoost initialized                                                            
Fitting model                                                                   
Loss: -0.5                                                                      
CatBoost initialized                                                            
Fitting model                                                                   
Loss: -0.5                                                                      
CatBoost initialized                                                            
Fitting model                                                                   
Loss: -0.5                                                                      
Ca

KeyboardInterrupt: 

In [ ]:
evaluation_accuracy